In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

In [4]:
df1 = pd.read_excel("/content/drive/MyDrive/Tacos_data/MIZO--- ENGLISH---- MIZO.xlsx")

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       800 non-null    object
 1   Sentiment  800 non-null    object
dtypes: object(2)
memory usage: 12.6+ KB


In [6]:
df1.head()

,Text,Sentiment
0,"He video hi en suh, hun khawhralna a ni.",NEGATIVE
1,I video te hi ka duh lo khawp mai.,NEGATIVE
2,i video ah hian information mak tak tak i nei a.,NEGATIVE
3,Engvangin nge i video hi a chhiat em em?,NEGATIVE
4,I video te hi a tha tawk ka ti lo,NEGATIVE


In [7]:
df2 = pd.read_excel("/content/drive/MyDrive/Tacos_data/English_Tacos_Train.xlsx")

In [8]:
df2.head()

,Text,Sentiment
0,Basically there's a family where a little boy ...,NEGATIVE
1,"This show was an amazing, fresh & innovative i...",NEGATIVE
2,Encouraged by the positive comments about this...,NEGATIVE
3,Phil the Alien is one of those quirky films wh...,NEGATIVE
4,I saw this movie when I was about 12 when it c...,NEGATIVE


In [9]:
#df = df1

In [10]:
df =  pd.concat([df1, df2])

In [11]:
df.head()

,Text,Sentiment
0,"He video hi en suh, hun khawhralna a ni.",NEGATIVE
1,I video te hi ka duh lo khawp mai.,NEGATIVE
2,i video ah hian information mak tak tak i nei a.,NEGATIVE
3,Engvangin nge i video hi a chhiat em em?,NEGATIVE
4,I video te hi a tha tawk ka ti lo,NEGATIVE


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1600 entries, 0 to 799
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       1600 non-null   object
 1   Sentiment  1600 non-null   object
dtypes: object(2)
memory usage: 37.5+ KB


In [13]:
#converting the labels to numbers
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['labels'] = le.fit_transform(df.Sentiment.values)

In [14]:
df.head()

,Text,Sentiment,labels
0,"He video hi en suh, hun khawhralna a ni.",NEGATIVE,0
1,I video te hi ka duh lo khawp mai.,NEGATIVE,0
2,i video ah hian information mak tak tak i nei a.,NEGATIVE,0
3,Engvangin nge i video hi a chhiat em em?,NEGATIVE,0
4,I video te hi a tha tawk ka ti lo,NEGATIVE,0


In [15]:
from sklearn.utils import shuffle
df = shuffle(df)

In [16]:
positive_labels = (df['labels'] == 1).sum()
negative_labels = (df['labels'] == 0).sum()

In [17]:
print(f"Number of positive labels: {positive_labels}")
print(f"Number of negative labels: {negative_labels}")

Number of positive labels: 800
Number of negative labels: 800


In [18]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels_str, val_labels_str = train_test_split(list(df['Text']), list(df['Sentiment']), test_size=.2)

In [19]:
!pip install sentencepiece

In [20]:
!pip install transformers

In [21]:
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [22]:
!pip install torch==1.11.00.676617

ERROR: Could not find a version that satisfies the requirement torch==1.11.00.676617 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0)
ERROR: No matching distribution found for torch==1.11.00.676617


In [24]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
import torch

In [27]:
class spanish_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_labels = le.fit_transform(train_labels_str)
val_labels = le.transform(val_labels_str)

train_dataset = spanish_Dataset(train_encodings, train_labels)
val_dataset = spanish_Dataset(val_encodings, val_labels)

In [28]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {'Accuracy': acc,
            'F1': f1,
            'Precision': precision,
            'Recall': recall
           }

In [29]:
from transformers import EarlyStoppingCallback, IntervalStrategy,Trainer, TrainingArguments

In [30]:
!pip install --upgrade accelerate

In [31]:
!pip install accelerate -U

In [32]:
!pip install transformers[torch]

In [33]:
!pip install accelerate>=0.21.0

In [34]:
!pip install --upgrade accelerate


In [35]:
import torch
torch.cuda.empty_cache()

In [36]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [37]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs 3,6,7,8
    per_device_train_batch_size=15,  # batch size per device during training
    per_device_eval_batch_size=15,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    do_eval=True,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    eval_steps=10
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset= val_dataset,
    compute_metrics=compute_metrics, # evaluation dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
10,0.690900,0.693541,0.496875,0.496875,0.496875,0.496875
20,0.697700,0.689296,0.543750,0.543750,0.543750,0.543750
30,0.688100,0.690400,0.540625,0.540625,0.540625,0.540625
40,0.691500,0.689834,0.540625,0.540625,0.540625,0.540625
50,0.691000,0.685954,0.540625,0.540625,0.540625,0.540625
60,0.673200,0.681080,0.537500,0.537500,0.537500,0.537500
70,0.629000,0.665637,0.543750,0.543750,0.543750,0.543750
80,0.628500,0.736991,0.506250,0.506250,0.506250,0.506250
90,0.707100,0.706499,0.556250,0.556250,0.556250,0.556250
100,0.594800,0.651206,0.621875,0.621875,0.621875,0.621875


TrainOutput(global_step=430, training_loss=0.4355234337407489, metrics={'train_runtime': 1097.9866, 'train_samples_per_second': 5.829, 'train_steps_per_second': 0.392, 'total_flos': 1683910754304000.0, 'train_loss': 0.4355234337407489, 'epoch': 5.0})

In [38]:
metrics=trainer.evaluate()

In [39]:
metrics

{'eval_loss': 1.0236341953277588,
 'eval_Accuracy': 0.703125,
 'eval_F1': 0.703125,
 'eval_Precision': 0.703125,
 'eval_Recall': 0.703125,
 'eval_runtime': 11.7441,
 'eval_samples_per_second': 27.248,
 'eval_steps_per_second': 1.873,
 'epoch': 5.0}

In [40]:
test_df = pd.read_excel("/content/drive/MyDrive/Tacos_data/mizo_data_tacos_test_final(1).xlsx")

In [41]:
len(test_df)

200

In [42]:
test_df.head()

,Text,Sentiment
0,"He hla hi hmangaihna lamah te, min kawngah te ...",POSITIVE
1,He track-a guitar hnathawh inzawmkhawm leh sou...,POSITIVE
2,Genre hrang hrang inzawmkhawmna famkim ani . H...,POSITIVE
3,"A hla thu hi hla thu a ni a, a thluk pawh a ma...",POSITIVE
4,Positivity pholang thei hla tichaktu. Artist-i...,POSITIVE


In [43]:
test_texts = list(test_df['Text'])

In [44]:
test_labels = le.transform(test_df['Sentiment'])

In [45]:
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
test_dataset = spanish_Dataset(test_encodings, test_labels)

In [46]:
# here we are evaluating a model using the Trainer object trainer on a test dataset.
test_metrics = trainer.evaluate(test_dataset)

In [47]:
test_metrics

{'eval_loss': 1.0628596544265747,
 'eval_Accuracy': 0.725,
 'eval_F1': 0.7250000000000001,
 'eval_Precision': 0.725,
 'eval_Recall': 0.725,
 'eval_runtime': 7.2032,
 'eval_samples_per_second': 27.765,
 'eval_steps_per_second': 1.944,
 'epoch': 5.0}